In [68]:
from datasketch import MinHash, MinHashLSH
import kshingle as ks
import numpy as np
import pandas as pd
import demoji
demoji.download_codes()
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

C:\Users\chile\AppData\Local\Temp\ipykernel_4276\3106212413.py:6: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chile\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chile\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [69]:
df_tweets = pd.read_csv('tweets_2022_abril_junio.csv')
df_tweets.shape

(4594980, 6)

In [70]:
df_tweets = df_tweets.drop_duplicates(subset='id')
df_tweets.shape

(4592806, 6)

In [71]:
df_tweets = df_tweets[df_tweets['text'].apply(lambda x: "RT @" not in x)].reset_index(drop=True)
df_tweets.shape

(1266850, 6)

In [72]:
stopwords_es = set(stopwords.words('spanish'))

In [73]:
def remove_emojis(texto):
    # Expresión regular para eliminar emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticonos
                               u"\U0001F300-\U0001F5FF"  # símbolos y pictogramas
                               u"\U0001F680-\U0001F6FF"  # transporte y símbolos de mapa
                               u"\U0001F1E0-\U0001F1FF"  # banderas de países
                               u"\U00002500-\U00002BEF"  # caracteres chinos, japoneses y coreanos (CJK)
                               u"\U00002702-\U000027B0"  # símbolos de negocio
                               u"\U00002702-\U000027B0"  # flechas y símbolos diversos
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    
    # Eliminar emojis
    texto_without_emojis = emoji_pattern.sub(r'', texto)
    
    # Preservar letras con tildes, la letra "ñ" y símbolos específicos
    texto_final = re.sub(r'[^a-zA-Zá-úÁ-ÚñÑ@/: ,.;-_]+', '', texto_without_emojis)
    
    return texto_final

In [74]:
def eliminar_stopwords_efficient(texto):
    tokens = word_tokenize(texto, language='spanish')
    texto_sin_stopwords = ' '.join([palabra for palabra in tokens if palabra.lower() not in stopwords_es]).lower()
    texto_sin_emojis_y_stopwords = remove_emojis(texto_sin_stopwords)
    return texto_sin_emojis_y_stopwords

In [75]:
df_tweets['text'] = df_tweets['text'].apply(eliminar_stopwords_efficient)

In [76]:
# Se eliminan los text que no tienen texto
df_tweets = df_tweets.dropna(subset=['text'])
df_tweets.shape

(1266850, 6)

In [80]:
sample = df_tweets.head(100000)

Se crean shingles

In [81]:
# Crear el shingle de cada tweet
sample['shingles'] = sample['text'].apply(lambda x: ks.shingleset_list(x, [5]))
sample

C:\Users\chile\AppData\Local\Temp\ipykernel_4276\1631585143.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['shingles'] = sample['text'].apply(lambda x: ks.shingleset_list(x, [5]))


,id,created_at,screen_name,text,favorite_count,retweet_count,shingles
0,1512186668387913732,2022-04-07 21:52:51 UTC,simonlodijo,@ unveranonaranja @ ruidosafest @ franciscamus...,0,0,"{@ fra, @ di, naran, ic @ , afest, @ die, nna..."
1,1512186985850544129,2022-04-07 21:54:07 UTC,MacaSimplemente,@ lasurivas @ elisaloncon @ siliconvalle @ val...,0,0,"{e @ v, a roj, as va, nata , n @ s, s @ e, ton..."
2,1512187189974683661,2022-04-07 21:54:55 UTC,LuisVer75699645,@ teanval @ izkia @ arturozunigaj excelente ...,0,0,"{zkia , lente, nigaj, eanva, ia @ , aj ex, exc..."
3,1512187226398076929,2022-04-07 21:55:04 UTC,MITERRED,@ mcubillossigall https : //t.co/gkgpwbzhz,0,0,"{ll ht, igall, mcub, https, : //, gkgpw, co/..."
4,1512187244248936452,2022-04-07 21:55:08 UTC,piaelizabethvm,@ simonlodijo @ ruidosafest @ franciscamusic @...,0,0,"{@ fra, @ di, ic @ , afest, o @ r, nnaro, sim..."
...,...,...,...,...,...,...,...
99995,1513530869322956801,2022-04-11 14:54:13 UTC,Drags18,@ baradit significa cagamos ... buenos dias ve...,0,0,"{nos d, orte , agamo, .. bu, , ru, siaet, ru..."
99996,1513563086871007232,2022-04-11 17:02:15 UTC,marcelohjleiva,@ baradit tongos mentiras calumnias medios des...,0,0,"{ nomb, as ca, umnia, edios, t ton, ongos, do ..."
99997,1513573497271816192,2022-04-11 17:43:37 UTC,ElBachValdivian,@ berfontaine cuantos merluzos saldrán decir m...,0,0,"{zos s, n dec, s sal, campa, ror ?, fonta, mpa..."
99998,1513560209519611904,2022-04-11 16:50:49 UTC,TitoSolari68,@ biobio @ horastvn @ chvnoticias @ lun vergen...,0,0,"{luir , @ ch, n ver, n @ c, icias, bio @, vno..."


In [82]:
# Crear un diccionario con key = id y value = shingles
dict_shingles = {}

for i in sample.index:
    id_tweet = sample["id"][i]
    shingles = sample["shingles"][i]
    dict_shingles[id_tweet] = shingles



In [84]:
# Se crea una lista de los usuarios
ids_tweet = list(dict_shingles.keys())
len(ids_tweet)

100000

In [85]:
dict_minhash = {}
for id_tweet in ids_tweet:
    shingles = dict_shingles[id_tweet]
    m = MinHash(num_perm=50)
    for d in shingles:
        m.update(d.encode('utf8'))
    dict_minhash[id_tweet] = m


In [91]:
# Ahora vemos el MinHashLSH
lsh = MinHashLSH(threshold=0.5, num_perm=50)
for id_tweet in ids_tweet:
    lsh.insert(id_tweet, dict_minhash[id_tweet])
    

In [92]:
sample_indexado = sample.set_index('id')

In [93]:
# Se crea una función para buscar los tweets similares
def buscar_tweets_similares(id_tweet, df_tweets, lsh, dict_minhash):
    # Se obtiene el MinHash del tweet
    m = dict_minhash[id_tweet]
    
    # Se obtienen los ids de los tweets similares
    ids_similares = lsh.query(m)
    
    # Se obtienen los tweets similares
    tweets_similares = df_tweets.loc[ids_similares]
    
    return tweets_similares


In [99]:
# Buscaremos quien escribe similar a un usuario en específico, para esto filtramos el dataframe por el usuario
user_name = 'MacaSimplemente'
df_user = sample_indexado[sample_indexado['screen_name'] == user_name]
df_user.head()

,created_at,screen_name,text,favorite_count,retweet_count,shingles
id,,,,,,
1512186985850544129,2022-04-07 21:54:07 UTC,MacaSimplemente,@ lasurivas @ elisaloncon @ siliconvalle @ val...,0,0,"{e @ v, a roj, as va, nata , n @ s, s @ e, ton..."
1512221405248585728,2022-04-08 00:10:53 UTC,MacaSimplemente,@ criordor @ camibasven gracias rechazo,0,0,"{r @ c, riord, acias, dor @, @ cam, chazo, iba..."
1512016957855305729,2022-04-07 10:38:29 UTC,MacaSimplemente,@ rkatrileo ja ja ja mitómana,0,0,"{ ja j, a ja , ómana, trile, rkat, atril, eo ..."
1512017519799726084,2022-04-07 10:40:43 UTC,MacaSimplemente,@ jaime_bassa puedes explicar canal contrataci...,0,0,"{ cont, res a, expl, trata, ataci, ión f, xpl..."
1513216223202357250,2022-04-10 18:03:56 UTC,MacaSimplemente,@ richarddcorobo @ jaime_bassa @ berfontaine c...,0,0,"{arddc, aro p, richa, e cla, rich, chard, tai..."


In [100]:
# iteramos sobre los tweets del usuario y buscamos los tweets similares, para esto usamos la función buscar_tweets_similares, guardamos las repeticiones en un diccionario de los usuarios similares, para saber quien es el usuario que más se repite
dict_similares = {}
for id_tweet in df_user.index:
    tweets_similares = buscar_tweets_similares(id_tweet, sample_indexado, lsh, dict_minhash)
    for id_similar in tweets_similares.index:
        if id_similar != id_tweet:
            screen_name = tweets_similares['screen_name'][id_similar]
            if screen_name in dict_similares:
                dict_similares[screen_name] += 1
            else:
                dict_similares[screen_name] = 1


In [102]:
# Obtener el usuario más similar 
max(dict_similares, key=dict_similares.get)

'pita09443405'